In [1]:
import numpy as np
import cv2
import time
import yaml
import os
from IPython.display import clear_output
import sys
import os.path
from deeplabcut.pose_estimation_tensorflow.nnet import predict
from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import data_to_input
import time
import pandas as pd
import numpy as np
import pickle
import os
import argparse
from pathlib import Path
from tqdm import tqdm
import tensorflow as tf
from deeplabcut.utils import auxiliaryfunctions
from skimage.util import img_as_ubyte
import numpy as np
from queue import Queue 
import StreamHandler as st
import deeplabcut as dlc

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TKAgg')
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))


In [2]:
train_vid_dir = '/home/nickt/Therahome/Videos/Training/'
main_dir = '/home/nickt/Therahome/'
pcf = '/home/nickt/Therahome/Therahome-Nick-2021-05-12/config.yaml'

os.chdir(main_dir)
vid_dir = os.path.join(main_dir, 'Videos/Training')

In [3]:
stream = st.StreamHandler([2], classify_behavior=True, 
                          behavior_model_path=os.path.join(main_dir, 'SVM_BasicMovement.sav'))



In [4]:
stream.beginCapture(pcf ,labelVideo=True, shuffle =1)

/home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1/test/pose_cfg.yaml
Using snapshot-810000 for model /home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1
Initializing MobileNet
INFO:tensorflow:Restoring parameters from /home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1/train/snapshot-810000


INFO:tensorflow:Restoring parameters from /home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1/train/snapshot-810000
/home/nickt/Therahome/StreamHandler.py:589: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  tmpcfg = yaml.load(config)
/home/nickt/Therahome/StreamHandler.py:591: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  tmpcfg = yaml.load(stream)


[array([2.])]###STOPPING###
Stopping


In [106]:
files = [os.path.join(vid_dir, file) for file in os.listdir(vid_dir) if file.endswith('h5')]
ids = {'Contracted': 1, 
       'Straight': 0, 
       'Extended': 2}

y = np.empty(0)
x = []
for file in files:
    value = [ids[val] for val in ids.keys() if val in file][0]
    data = pd.read_hdf(file)
    numel = len(data)
    y = np.concatenate((y, np.ones(numel) * value), axis = 0)
    if len(x) == 0:
        x = data
    else:
        x = np.concatenate((x, data), axis=0)

np.shape()

(75,)

In [2]:
def test_model(input_data, output, count = 3, print_loop=True, print_mean = True, get_meta = False, get_model = False):
    scores = []
    meta = {}
    for i in range(count):
        
        
        #x_train, x_test, y_train, y_test = train_test_split(input_data, output, test_size = .3)
        
        count = len(output)
        random = np.random.rand(count)
        tmpidx = np.linspace(0,count-1,count, dtype = int)
        idx = tmpidx[np.argsort(random)]
        split_prop = .3 #test set percentage
        split_idx = int(len(idx) * split_prop)

        test_idx = idx[0:split_idx]
        train_idx = idx[split_idx:]
        x_test = input_data[test_idx]
        x_train = input_data[train_idx]
        print(len(x_test), len(x_train))
        y_test = output[test_idx]
        y_train = output[train_idx]
        
        
        clf = svm.SVC(kernel='linear')
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        scores.append(metrics.accuracy_score(y_test, y_pred))
        
        y = clf.decision_function(x_test)
        w_norm = np.linalg.norm(clf.coef_)
        dist = y / w_norm
        
        if print_loop:
            print("Accuracy {}:".format(i), metrics.accuracy_score(y_test, y_pred))
        tmp_meta = {}
        if get_meta or get_model:
            tmp_meta['x_train'] = x_train
            tmp_meta['y_train'] = y_train
            tmp_meta['x_test'] = x_test
            tmp_meta['y_test'] = y_test
            tmp_meta['y_pred'] = y_pred
            tmp_meta['score'] = np.mean(scores)
            tmp_meta['dist'] = dist
            tmp_meta['split_idx'] = split_idx
            tmp_meta['test_idx'] = test_idx
            tmp_meta['train_idx'] = train_idx
        meta[str(i)] = tmp_meta
    if print_mean:
        print("Mean Accuracy", np.mean(scores))
    if get_model:
        return scores, meta, clf
    elif get_meta:
        return scores, meta
    else:
        return scores

    

    

In [144]:
s, meta, clf = test_model(x, y, 1, get_model=True)
filename = os.path.join(main_dir, 'SVM_BasicMovement.sav')
pickle.dump(clf, open(filename, 'wb'))


317 742
Accuracy 0: 0.9968454258675079
Mean Accuracy 0.9968454258675079


In [149]:
%%timeit
clf.predict(x[0].reshape(1,-1))

56.4 µs ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [150]:
a = [1,2,3]
str(a)

'[1, 2, 3]'

In [124]:
np.shape(meta['0']['x_train'])[0]

742

In [147]:
a = 'b'
not a == None


True

/home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1/test/pose_cfg.yaml
Using snapshot-810000 for model /home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1
Initializing MobileNet


In [6]:
vids = [os.path.join(train_vid_dir, vid) for vid in os.listdir(train_vid_dir) if vid.endswith('avi')]

project_name = 'Therahome'
experimenter = 'Nick'
project_created = len([file for file in os.listdir(main_dir) if file.startswith('{}-{}'.format(project_name, experimenter))]) != 0

if not project_created:
    dlc.create_new_project(project_name, experimenter, videos = vids, copy_videos='True', working_directory=main_dir)

['/home/nickt/Therahome/Videos/Training/ClosedHand_Extended.avi',
 '/home/nickt/Therahome/Videos/Training/ClosedHand_Contracted.avi',
 '/home/nickt/Therahome/Videos/Training/ClosedHand_Straight.avi']

In [ ]:
dlc.extract_frames(pcf, algo='uniform' )

In [ ]:
dlc.label_frames(pcf)

In [ ]:
dlc.create_training_dataset(pcf)

In [ ]:
dlc.train_network(pcf, displayiters=10000, saveiters=5000)

In [ ]:
dlc.evaluate_network(pcf)

In [5]:
dlc.analyze_videos(pcf, vids)

Using snapshot-810000 for model /home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1
Initializing MobileNet
INFO:tensorflow:Restoring parameters from /home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1/train/snapshot-810000


INFO:tensorflow:Restoring parameters from /home/nickt/Therahome/Therahome-Nick-2021-05-12/dlc-models/iteration-0/TherahomeMay12-trainset95shuffle1/train/snapshot-810000
  0%|          | 0/424 [00:00<?, ?it/s]

Starting to analyze %  /home/nickt/Therahome/Videos/Training/ClosedHand_Extended.avi
Loading  /home/nickt/Therahome/Videos/Training/ClosedHand_Extended.avi
Duration of video [s]:  14.13 , recorded with  30.0 fps!
Overall # of frames:  424  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


430it [00:11, 53.27it/s]                         
  0%|          | 0/295 [00:00<?, ?it/s]

Detected frames:  424
Saving results in /home/nickt/Therahome/Videos/Training...
Starting to analyze %  /home/nickt/Therahome/Videos/Training/ClosedHand_Contracted.avi
Loading  /home/nickt/Therahome/Videos/Training/ClosedHand_Contracted.avi
Duration of video [s]:  9.83 , recorded with  30.0 fps!
Overall # of frames:  295  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


300it [00:05, 50.83it/s]                         
  0%|          | 0/340 [00:00<?, ?it/s]

Detected frames:  295
Saving results in /home/nickt/Therahome/Videos/Training...
Starting to analyze %  /home/nickt/Therahome/Videos/Training/ClosedHand_Straight.avi
Loading  /home/nickt/Therahome/Videos/Training/ClosedHand_Straight.avi
Duration of video [s]:  11.33 , recorded with  30.0 fps!
Overall # of frames:  340  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


350it [00:07, 45.44it/s]                         


Detected frames:  340
Saving results in /home/nickt/Therahome/Videos/Training...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DLC_mobnet_100_TherahomeMay12shuffle1_810000'

In [8]:
dlc.create_labeled_video(pcf, vids)

  4%|▎         | 15/424 [00:00<00:02, 139.02it/s]

Starting %  /home/nickt/Therahome/Videos/Training ['/home/nickt/Therahome/Videos/Training/ClosedHand_Extended.avi', '/home/nickt/Therahome/Videos/Training/ClosedHand_Contracted.avi', '/home/nickt/Therahome/Videos/Training/ClosedHand_Straight.avi']
Loading  /home/nickt/Therahome/Videos/Training/ClosedHand_Extended.avi and data.
424
Duration of video [s]:  14.13 , recorded with  30.0 fps!
Overall # of frames:  424 with cropped frame dimensions:  640 480
Generating frames and creating video.


  4%|▍         | 13/295 [00:00<00:02, 125.71it/s]

Starting %  /home/nickt/Therahome/Videos/Training ['/home/nickt/Therahome/Videos/Training/ClosedHand_Extended.avi', '/home/nickt/Therahome/Videos/Training/ClosedHand_Contracted.avi', '/home/nickt/Therahome/Videos/Training/ClosedHand_Straight.avi']
Loading  /home/nickt/Therahome/Videos/Training/ClosedHand_Contracted.avi and data.
295
Duration of video [s]:  9.83 , recorded with  30.0 fps!
Overall # of frames:  295 with cropped frame dimensions:  640 480
Generating frames and creating video.


  3%|▎         | 11/340 [00:00<00:03, 101.89it/s]

Starting %  /home/nickt/Therahome/Videos/Training ['/home/nickt/Therahome/Videos/Training/ClosedHand_Extended.avi', '/home/nickt/Therahome/Videos/Training/ClosedHand_Contracted.avi', '/home/nickt/Therahome/Videos/Training/ClosedHand_Straight.avi']
Loading  /home/nickt/Therahome/Videos/Training/ClosedHand_Straight.avi and data.
340
Duration of video [s]:  11.33 , recorded with  30.0 fps!
Overall # of frames:  340 with cropped frame dimensions:  640 480
Generating frames and creating video.


100%|██████████| 340/340 [00:02<00:00, 146.49it/s]
